In [1]:
from Python.tigre.utilities.io.VarianDataLoader import read_varian_geometry, get_xmlns, load_blank_projections, load_projections, calculate_angular_threshold
from Python.tigre.utilities.io.xim_io import XIM
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from tigre.utilities.geometry import Geometry
import xml.etree.ElementTree as ET
import glob

In [2]:
mydir = "C:/CBCT_Scatter_Removal_Project/data/clinical/4100758/2021-07-06_101037/11528a87-b894-4239-846d-45d8fd1efeda"

In [3]:
geometry, scan_params = read_varian_geometry(filepath=mydir)

In [4]:
ns = get_xmlns(scan_params)
rot_velocity = float(scan_params.find("Acquisitions/Velocity", ns).text)
frame_rate = rot_velocity = float(scan_params.find("Acquisitions/FrameRate", ns).text)
angular_threshold = calculate_angular_threshold(rot_velocity, frame_rate)

In [5]:
p,a,air = load_projections(mydir,calculate_angular_threshold(rot_velocity,frame_rate))
bp,ba,bair = load_blank_projections(mydir,scan_params)

100%|██████████| 900/900 [02:48<00:00,  5.35it/s]


In [7]:
projs, angles, airnorms = p,a,air
blank_projs, blank_angles, blank_airnorms = bp,ba,bair

In [ ]:
log_projs = np.zeros(np.size(projs))
eps = np.finfo(projs.dtype).eps
if blank_angles.size==0:
    for i in range(len(angles)):
        cf_air = airnorms[i]/blank_airnorms
        log_projs[i] = np.log(cf_air*blank_projs/(projs[i]+eps)+eps)
else:
    # TODO: sort to be monotonically increasing
    i_sort = np.argsort(blank_angles % 360)
    blank_angles = blank_angles[i_sort]
    blank_projs = blank_projs[i_sort,]
    blank_airnorms = blank_airnorms[i_sort]

    indices = np.linspace(0,len(blank_angles)-1,len(blank_angles))
    for i in range(len(angles)):
        y = np.interp(angles[i],indices,blank_angles)
        


In [ ]:
def interp_weight(x,xp,N=360):
    # linear interpolation of xp at x, mod(N). 
    # xp must be monotonically increasing.
    i_min = np.argmin(abs(x-xp))
    if x-xp[i_min]>=0:
        i_lower=i_min
    else:
        i_lower = (i_min-1)%len(xp)

    i_upper = (i_lower+1)%len(xp)

    xp_diff = (xp[i_upper]-xp[i_lower])%N
    weights = 1-(np.array([(xp[i_upper]-x), (x-xp[i_lower])])%N)/float(xp_diff) 
    return i_lower, weights

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [ ]:
# TODO: verify this works properly
N = 360
xp = N*np.linspace(0,0.8,6)
x = 358%N
id = np.argmin(abs(xp-x))

d = (x-xp[id])

if d>=0:
    i_lower=id
else:
    i_lower = id-1
i_upper = (i_lower+1)%len(xp)

xp_diff = (xp[i_upper]-xp[i_lower] )%N

weights = 1-(np.array([(xp[i_upper]-x), (x-xp[i_lower])])%N)/float(xp_diff) 
